# 一、数据读取

In [33]:
import pandas as pd
data=pd.read_excel('RFM聚类分析.xlsx',index_col='用户编码')
data.head()

,首次投资时间,最近一次投资时间,总计投资总金额,总计投标总次数
用户编码,,,,
45B3CCE7-957B-4D54-9626-6D62731D119B,2016-06-25,2016-06-26,20000.0,3
9921C535-75FC-4695-8551-32B8D717B4A5,2016-06-26,2016-06-26,50000.0,3
2F00D4E3-CC7D-4DB1-B210-6F095156C5B8,2016-06-13,2016-06-19,108000.0,4
150218D8-BF32-406D-A3A8-8A19CB1E85AA,2016-06-16,2016-06-16,30000.0,3
98C26D4A-708F-4334-B3D7-EDAE9FD240C8,2016-06-27,2016-06-27,100000.0,3


# 二、数据探索及预处理

In [4]:
data.describe(include='all')

,首次投资时间,最近一次投资时间,总计投资总金额,总计投标总次数
count,200,200,200.000000,200.00000
unique,24,38,NaN,NaN
top,2016-06-22 00:00:00,2016-06-25 00:00:00,NaN,NaN
freq,20,25,NaN,NaN
first,2016-06-06 00:00:00,2016-06-06 00:00:00,NaN,NaN
last,2016-06-29 00:00:00,2016-07-18 00:00:00,NaN,NaN
mean,NaN,NaN,47223.409500,5.92000
std,NaN,NaN,43381.496372,4.84883
min,NaN,NaN,10100.000000,2.00000
25%,NaN,NaN,20000.000000,3.00000


In [5]:
#数据提取日为2016/07/20
from datetime import datetime
ex_data=datetime(2016,7,20)


In [6]:
#R:最近一次投资日期和提数日的天数差
diff_days=ex_data-data['最近一次投资时间']
R=[]
for i in diff_days:
    R.append(i.days)
R[:5]

[24, 24, 31, 34, 23]

In [9]:
#用户在投月数
from math import ceil
diff=ex_data-data['首次投资时间']
diff_months=[]
for i in diff:
    diff_months.append(ceil(i.days/30))
diff_months[:5]

[1, 1, 2, 2, 1]

In [12]:
#F月平均投资次数
F=(data['总计投标总次数']/diff_months).values

In [13]:
F[:5]

array([3. , 3. , 2. , 1.5, 3. ])

In [14]:
#M月平均投资金额
M=(data['总计投资总金额']/diff_months).values
M[:5]

array([ 20000.,  50000.,  54000.,  15000., 100000.])

# 三、RFM聚类分析

In [17]:
cdata=pd.DataFrame([R,list(F),list(M)]).T

In [19]:
cdata.columns=['R','F','M']
cdata.index=data.index
cdata.head()

,R,F,M
用户编码,,,
45B3CCE7-957B-4D54-9626-6D62731D119B,24.0,3.0,20000.0
9921C535-75FC-4695-8551-32B8D717B4A5,24.0,3.0,50000.0
2F00D4E3-CC7D-4DB1-B210-6F095156C5B8,31.0,2.0,54000.0
150218D8-BF32-406D-A3A8-8A19CB1E85AA,34.0,1.5,15000.0
98C26D4A-708F-4334-B3D7-EDAE9FD240C8,23.0,3.0,100000.0


数据标准化

In [20]:
zcdata=(cdata-cdata.mean())/cdata.std()

In [21]:
zcdata.head()

,R,F,M
用户编码,,,
45B3CCE7-957B-4D54-9626-6D62731D119B,0.140422,-0.353751,-0.392133
9921C535-75FC-4695-8551-32B8D717B4A5,0.140422,-0.353751,0.351423
2F00D4E3-CC7D-4DB1-B210-6F095156C5B8,0.876714,-0.638461,0.450564
150218D8-BF32-406D-A3A8-8A19CB1E85AA,1.192268,-0.780815,-0.516059
98C26D4A-708F-4334-B3D7-EDAE9FD240C8,0.035237,-0.353751,1.590683


In [22]:
#sklearn
from sklearn.cluster import KMeans
kmodel=KMeans(n_clusters=4,n_jobs=4,max_iter=100,random_state=0)
kmodel.fit(zcdata)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=4, n_init=10, n_jobs=4, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [24]:
#统计每个类别的频率
pd.Series(kmodel.labels_).value_counts()

0    128
2     44
1     19
3      9
dtype: int64

In [27]:
#把类别赋回到原数据cdata
cdata_rst=pd.concat([cdata,pd.Series(kmodel.labels_,index=cdata.index)],axis=1)

In [28]:
#命名最后一列为类别
cdata_rst.columns=list(cdata.columns)+['类别']

In [29]:
cdata_rst.head()

,R,F,M,类别
用户编码,,,,
45B3CCE7-957B-4D54-9626-6D62731D119B,24.0,3.0,20000.0,0
9921C535-75FC-4695-8551-32B8D717B4A5,24.0,3.0,50000.0,0
2F00D4E3-CC7D-4DB1-B210-6F095156C5B8,31.0,2.0,54000.0,0
150218D8-BF32-406D-A3A8-8A19CB1E85AA,34.0,1.5,15000.0,0
98C26D4A-708F-4334-B3D7-EDAE9FD240C8,23.0,3.0,100000.0,1


In [31]:
#按照类别分类统计R、F、M的均值
cdata_rst.groupby(cdata_rst['类别']).mean()

,R,F,M
类别,,,
0,27.859375,2.820312,21906.754297
1,20.684211,4.552632,115842.105263
2,10.568182,5.579545,26984.313636
3,12.111111,17.277778,107986.000000


结论：
类别3：R、F、M都比较高，属于重要价值客户
类别0：R、F、M比较低，属于低价值客户
类别1：R比较低，F还好，M比较高，属于重要保持客户